<a href="https://colab.research.google.com/github/kiakass/blog/blob/main/embedding_blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/text/guide/word_embeddings <br>
https://wikidocs.net/32105<br>

https://keras.io/ko/layers/embeddings/

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from scipy.sparse import csr_matrix, coo_matrix

In [ ]:
movies_fn = ('https://raw.githubusercontent.com/kiakass/share/master/movies.csv')   # 9,125 건, 
ratings_fn = ('https://raw.githubusercontent.com/kiakass/share/master/ratings.csv') # 100,004 건
users_fn =('https://raw.githubusercontent.com/kiakass/share/master/user_features.csv')

movies=pd.read_csv(movies_fn).fillna(0)
ratings=pd.read_csv(ratings_fn).fillna(0)
users=pd.read_csv(users_fn).fillna(0)

In [ ]:
std = StandardScaler()
ratings= minmax_scale(ratings)

In [ ]:
ratings.shape[0]

100004

## Embedding

- input_dim 이 unique한 item수보다 많아야함

In [ ]:
test_embedding = Embedding(input_dim=4, output_dim=2)
test_embedding(tf.constant([1,2,3,3]))

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.01988821,  0.03895975],
       [-0.02904925,  0.03168467],
       [ 0.039296  ,  0.00257453],
       [ 0.039296  ,  0.00257453]], dtype=float32)>

In [ ]:
tf.constant([[1,2,3],[4,5,3],[4,5,3]]).shape

TensorShape([3, 3])

In [ ]:
test_embedding = Embedding(input_dim=6, output_dim=2)
print(test_embedding(tf.constant([[1,2,3],[4,5,3],[4,5,3]]))) #dim : 1,2,3,4,5 + 1

tf.Tensor(
[[[ 0.03200975  0.04296975]
  [ 0.03023063 -0.00410701]
  [ 0.03850274  0.0393835 ]]

 [[-0.0034255   0.01435343]
  [-0.00296481  0.0394582 ]
  [ 0.03850274  0.0393835 ]]

 [[-0.0034255   0.01435343]
  [-0.00296481  0.0394582 ]
  [ 0.03850274  0.0393835 ]]], shape=(3, 3, 2), dtype=float32)


In [ ]:
# data shuffle, sampling 10s 
np.random.seed(2021)
df=ratings.sample(frac=1).iloc[:10,:3].sort_values(by='userId').reset_index(drop=True)
df

,userId,movieId,rating
0,19,361,4.0
1,150,1668,2.5
2,195,2015,4.0
3,200,36529,5.0
4,232,1073,3.0
5,547,7946,1.5
6,569,377,4.0
7,581,31374,0.5
8,595,2059,3.0
9,624,142448,3.0


In [ ]:
data_embedding = Embedding(df.shape[0], 2)
data_embedding(np.array(df))

InvalidArgumentError: ignored

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['userId_l'] = le.fit_transform(df['userId'])
df['movieId_l'] = le.fit_transform(df['movieId'])
df['rating_l'] = le.fit_transform(df['rating'])
np.c_[le.inverse_transform(le.fit_transform(df['userId'])),df['userId_l']]

array([[ 19,   0],
       [150,   1],
       [195,   2],
       [200,   3],
       [232,   4],
       [547,   5],
       [569,   6],
       [581,   7],
       [595,   8],
       [624,   9]])

In [ ]:
df

,userId,movieId,rating,userId_l,movieId_l,rating_l
0,19,361,4.0,0,0,4
1,150,1668,2.5,1,3,2
2,195,2015,4.0,2,4,4
3,200,36529,5.0,3,8,5
4,232,1073,3.0,4,2,3
5,547,7946,1.5,5,6,1
6,569,377,4.0,6,1,4
7,581,31374,0.5,7,7,0
8,595,2059,3.0,8,5,3
9,624,142448,3.0,9,9,3


In [ ]:
data_embedding = Embedding(df.shape[0], 3)
print(data_embedding(np.array(df.iloc[:,3:])))

tf.Tensor(
[[[-0.00356021 -0.03878094 -0.01316022]
  [-0.00356021 -0.03878094 -0.01316022]
  [ 0.00570418  0.0359746  -0.01192591]]

 [[-0.01637206 -0.00218325  0.04531309]
  [-0.0449886   0.02501824 -0.0188338 ]
  [-0.04195038 -0.00319714 -0.00181288]]

 [[-0.04195038 -0.00319714 -0.00181288]
  [ 0.00570418  0.0359746  -0.01192591]
  [ 0.00570418  0.0359746  -0.01192591]]

 [[-0.0449886   0.02501824 -0.0188338 ]
  [-0.03974012 -0.0132104   0.02488993]
  [-0.00099941 -0.01170168 -0.04620849]]

 [[ 0.00570418  0.0359746  -0.01192591]
  [-0.04195038 -0.00319714 -0.00181288]
  [-0.0449886   0.02501824 -0.0188338 ]]

 [[-0.00099941 -0.01170168 -0.04620849]
  [ 0.03029737  0.03570536 -0.04685453]
  [-0.01637206 -0.00218325  0.04531309]]

 [[ 0.03029737  0.03570536 -0.04685453]
  [-0.01637206 -0.00218325  0.04531309]
  [ 0.00570418  0.0359746  -0.01192591]]

 [[ 0.02791201  0.01760424 -0.02436641]
  [ 0.02791201  0.01760424 -0.02436641]
  [-0.00356021 -0.03878094 -0.01316022]]

 [[-0.0397401

## COO matrix

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix

np.random.seed(2021)

ratings_fn = ('https://raw.githubusercontent.com/kiakass/share/master/ratings.csv') # 100,004 건
ratings=pd.read_csv(ratings_fn).fillna(0)

In [ ]:
# data shuffle, sampling 10s 
df=ratings.sample(frac=1).iloc[:10,:3].sort_values(by='userId').reset_index(drop=True)
df

,userId,movieId,rating
0,19,361,4.0
1,150,1668,2.5
2,195,2015,4.0
3,200,36529,5.0
4,232,1073,3.0
5,547,7946,1.5
6,569,377,4.0
7,581,31374,0.5
8,595,2059,3.0
9,624,142448,3.0


In [ ]:
df_matrix = df.pivot( index='userId',
                      columns='movieId',
                      values='rating').fillna(0)
df_matrix                      

movieId,361,377,1073,1668,2015,2059,7946,31374,36529,142448
userId,,,,,,,,,,
19,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
232,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0
569,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
595,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [ ]:
data = np.array(df['rating'])
row_idx = np.array(df['userId'])
col_idx = np.array(df['movieId'])

csr_rating_matrix = csr_matrix((data,(row_idx, col_idx)))
coo_rating_matrix = coo_matrix((data, (row_idx,col_idx)))
#
print(csr_rating_matrix,coo_rating_matrix, sep='\n\n')


  (19, 361)	4.0
  (150, 1668)	2.5
  (195, 2015)	4.0
  (200, 36529)	5.0
  (232, 1073)	3.0
  (547, 7946)	1.5
  (569, 377)	4.0
  (581, 31374)	0.5
  (595, 2059)	3.0
  (624, 142448)	3.0

  (19, 361)	4.0
  (150, 1668)	2.5
  (195, 2015)	4.0
  (200, 36529)	5.0
  (232, 1073)	3.0
  (547, 7946)	1.5
  (569, 377)	4.0
  (581, 31374)	0.5
  (595, 2059)	3.0
  (624, 142448)	3.0


In [ ]:
# data shuffle, sampling 20s 
df=users.sample(frac=1).reset_index(drop=True).iloc[:10,:3].sort_values(by='userId')
df

,userId,gender,age
3,13,MAIL,20
2,93,MAIL,40
5,132,MAIL,20
4,237,MAIL,40
1,396,FEMAIL,50
9,401,FEMAIL,20
7,439,FEMAIL,20
6,648,MAIL,50
0,661,MAIL,40
8,664,MAIL,20


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['userId_l'] = le.fit_transform(df['userId'])
df['gender_l'] = le.fit_transform(df['gender'])
df['age_l'] = le.fit_transform(df['age'])
np.c_[df['userId_l'],le.inverse_transform(le.fit_transform(df['userId']))]

array([[  0,  13],
       [  1,  93],
       [  2, 132],
       [  3, 237],
       [  4, 396],
       [  5, 401],
       [  6, 439],
       [  7, 648],
       [  8, 661],
       [  9, 664]])

In [ ]:
df

,userId,gender,age,userId_l,gender_l,age_l
3,13,MAIL,20,0,1,0
2,93,MAIL,40,1,1,1
5,132,MAIL,20,2,1,0
4,237,MAIL,40,3,1,1
1,396,FEMAIL,50,4,0,2
9,401,FEMAIL,20,5,0,0
7,439,FEMAIL,20,6,0,0
6,648,MAIL,50,7,1,2
0,661,MAIL,40,8,1,1
8,664,MAIL,20,9,1,0


In [ ]:
df_matrix = df.pivot( index='userId_l',
                      columns='movieId',
                      values='rating').fillna(0)

## Word tokenizing

In [ ]:
tokenized_text = [['Hope', 'to', 'see', 'you', 'soon'], ['Nice', 'to', 'see', 'you', 'again']]

In [ ]:
le = LabelEncoder()
le_tmp=le.fit_transform(np.r_[tokenized_text[0],tokenized_text[1]])
le_tmp

array([0, 5, 3, 6, 4, 1, 5, 3, 6, 2])

In [ ]:
le_text = [le_tmp[:len(tokenized_text[0])].tolist(),le_tmp[-len(tokenized_text[1]):].tolist()]

In [ ]:
le_text = [le_tmp[:5].tolist(), le_tmp[5:].tolist()]
le_text

[[0, 5, 3, 6, 4], [1, 5, 3, 6, 2]]

In [ ]:
le_embedding = Embedding(input_dim=7, output_dim=2)
le_embedding(np.array(le_text))

<tf.Tensor: shape=(2, 5, 2), dtype=float32, numpy=
array([[[-0.03124826, -0.01618547],
        [-0.00946225, -0.04395205],
        [ 0.00861239,  0.03872142],
        [-0.02144307,  0.01548406],
        [ 0.04849822, -0.02543857]],

       [[ 0.04483184,  0.01845212],
        [-0.00946225, -0.04395205],
        [ 0.00861239,  0.03872142],
        [-0.02144307,  0.01548406],
        [ 0.03288463, -0.0261355 ]]], dtype=float32)>

In [ ]:
le.inverse_transform(le_token)

array(['Hope', 'to', 'see', 'you', 'soon', 'Nice', 'to', 'see', 'you',
       'again'], dtype='<U5')